# Linear Regression

This notebook implement linear regression on the data set


## 0 Load Libraries and Data

Import the necessary libraries and load the dataset.

In [22]:
# Linear Regression for ESG Score Prediction

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import pickle
import os
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import StratifiedKFold
import time

# Define a dictionary to store model metrics
model_metrics = {}

# Create an output directory for our results
os.makedirs('strat_linreg_eval_metrics', exist_ok=True)

# After running all your models and having model_metrics dictionary populated
from thesis_visualizations import visualize_complete_results

# 1. Data Loading and Preparation

In [23]:
# 1.1. Set up data paths
class Location:
    """
    Helper class for managing file paths.
    """
    def __init__(self, base_dir):
        """
        Initialize with the base directory.
        """
        self.base_dir = base_dir
        
    def get_path(self, subdirectory, filename):
        """
        Get full path for a file in a subdirectory.
        """
        return os.path.join(self.base_dir, subdirectory, filename)

base_dir = os.getcwd()    
location = Location(base_dir)

In [24]:
# 1.2. Load column lists
subdirectory='pkl'
with open(location.get_path(subdirectory, 'base_columns.pkl'), 'rb') as f:
    base_columns = pickle.load(f)

with open(location.get_path(subdirectory, 'yeo_columns.pkl'), 'rb') as f:
    yeo_columns = pickle.load(f)

print("Base Columns (sample):", base_columns[:5])
print("Yeo Columns (sample):", yeo_columns[:5])

Base Columns (sample): ['market_cap_usd', 'net_income_usd', 'hist_pe', 'hist_book_px', 'hist_fcf_yld']
Yeo Columns (sample): ['yeo_joh_market_cap_usd', 'yeo_joh_net_income_usd', 'yeo_joh_hist_pe', 'yeo_joh_hist_book_px', 'yeo_joh_hist_fcf_yld']


In [25]:
# 1.3. Load feature and target data
subdirectory = 'data'
feature_path = location.get_path(subdirectory, 'combined_df_for_ml_models.csv')
feature_df = pd.read_csv(feature_path, index_col='issuer_name')
feature_df = feature_df.convert_dtypes()
feature_df.sort_index(ascending=True, inplace=True)

score_path = location.get_path(subdirectory, 'score.csv')
score_df = pd.read_csv(score_path, index_col='issuer_name')
score_df = score_df.convert_dtypes()
score_df.sort_index(ascending=True, inplace=True)

In [26]:
print("Feature data shape:", feature_df.shape)
print("Score data shape:", score_df.shape)
print("\nScore data preview:")
score_df.head()

Feature data shape: (2202, 388)
Score data shape: (2202, 1)

Score data preview:


,esg_score
issuer_name,
10X Genomics Inc,4.6
3I GROUP PLC,9.7
3M COMPANY,9.5
A. O. SMITH CORPORATION,4.8
A.P. MOELLER - MAERSK A/S,7.6


In [27]:
# 1.4. Create feature sets and target variable
LR_Base = feature_df[base_columns]
LR_Yeo = feature_df[yeo_columns]
y = score_df
print("Base feature data shape:", LR_Base.shape)
print("Yeo data shape:", LR_Yeo.shape)



Base feature data shape: (2202, 362)
Yeo data shape: (2202, 362)


In [28]:
# Create copies with random variables added
LR_Base_random = LR_Base.copy()
LR_Yeo_random = LR_Yeo.copy()

# Add random variables to the copies
np.random.seed(42)  # For reproducibility
LR_Base_random['random_feature'] = np.random.normal(0, 1, size=LR_Base.shape[0])
LR_Yeo_random['random_feature'] = np.random.normal(0, 1, size=LR_Yeo.shape[0])

print("Created new datasets with random features")
print(f"X_base_random shape: {LR_Base_random.shape}")
print(f"X_yeo_random shape: {LR_Yeo_random.shape}")

Created new datasets with random features
X_base_random shape: (2202, 363)
X_yeo_random shape: (2202, 363)


# 2. Model definition

## 2.1 Stratified shuffle function

In [29]:
from sklearn.model_selection import StratifiedShuffleSplit

def perform_stratified_split_by_sector(X, y, test_size=0.2, random_state=42):
    """
    Performs a stratified train-test split based on GICS sectors.
    
    Parameters:
    -----------
    X : pandas.DataFrame
        Feature dataframe
    y : pandas.DataFrame or Series
        Target variable (ESG scores)
    test_size : float, default=0.2
        Proportion of the dataset to include in the test split
    random_state : int, default=42
        Random seed for reproducibility
        
    Returns:
    --------
    X_train, X_test, y_train, y_test : Dataframes split while preserving sector proportions
    """
    # Extract sector columns - these start with 'gics_sector_'
    sector_columns = [col for col in X.columns if col.startswith('gics_sector_')]
    
    # Create a sector label for each company (convert one-hot to single label)
    # We need a single column for stratification
    sector_data = X[sector_columns].copy()
    sector_labels = np.zeros(len(X), dtype=int)
    
    for i, col in enumerate(sector_columns):
        # Assign a unique integer to each sector
        sector_labels[sector_data[col] == 1] = i
    
    # Initialize stratified split
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    
    # Get train and test indices preserving sector proportions
    for train_idx, test_idx in sss.split(X, sector_labels):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    # Verify sector distributions are maintained
    print("Sector distribution check:")
    for i, col in enumerate(sector_columns):
        train_pct = X_train[col].mean() * 100
        test_pct = X_test[col].mean() * 100
        print(f"{col.replace('gics_sector_', '')}: Train {train_pct:.1f}%, Test {test_pct:.1f}%")
    
    return X_train, X_test, y_train, y_test

## 2.2 Linear model function

In [30]:
# Assume model_metrics is a global dict for storing summary results

def run_regression_model(X_data, y_data, model_name, 
                         model_type="linear", 
                         alpha=None, l1_ratio=None,
                         random_state=42, test_size=0.2):
    """
    Run regression on the provided data and evaluate performance.
    
    Parameters:
    -----------
    X_data : pandas.DataFrame
        Feature data
    y_data : pandas.Series or DataFrame
        Target variable
    model_name : str
        Identifier for the model
    model_type : str, default="linear"
        Type of model: "linear" or "elasticnet"
    alpha : float, optional
        Regularization strength (used if model_type == "elasticnet")
    l1_ratio : float, optional
        L1/L2 mixing ratio (used if model_type == "elasticnet")
    random_state : int, default=42
        Reproducibility seed
    test_size : float, default=0.2
        Proportion of data for testing
    """
    global model_metrics  # Use the global metrics dictionary
    
    # Use stratified split by sector
    X_train, X_test, y_train, y_test = perform_stratified_split_by_sector(
        X_data, y_data, test_size=test_size, random_state=random_state
    )

    # Select and initialize model
    if model_type == "linear":
        model = LinearRegression()
    elif model_type == "elasticnet":
        if alpha is None or l1_ratio is None:
            raise ValueError("ElasticNet requires both alpha and l1_ratio.")
        model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=random_state)
    else:
        raise ValueError(f"Unsupported model_type: {model_type}")

    # Fit and predict
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mse)

    # Logging
    print(f"\nModel: {model_name}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE : {mae:.4f}")
    print(f"  MSE : {mse:.4f}")
    print(f"  R²  : {r2:.4f}")

    model_metrics[model_name] = {
    'RMSE': rmse,
    'MAE': mae,
    'MSE': mse,
    'R2': r2,
    'n_companies': len(X_data),  # Total number of companies
    'n_companies_train': len(X_train),  # Companies in training set
    'n_companies_test': len(X_test),  # Companies in test set
    'model': model,
    'y_test': y_test,
    'y_pred': y_pred
}

## Sector specific linear model function

In [31]:
def run_sector_models(feature_df, score_df, base_columns, yeo_columns, 
                      LR_Base_random=None, LR_Yeo_random=None,
                      model_metrics_dict=None, random_state=42, test_size=0.2):
    """
    Run linear regression models separately for each GICS sector.
    
    Parameters:
    -----------
    feature_df : pandas.DataFrame
        Full feature dataframe
    score_df : pandas.Series or DataFrame
        Target variable (ESG scores)
    base_columns : list
        List of base feature columns
    yeo_columns : list
        List of Yeo-Johnson transformed feature columns
    LR_Base_random : pandas.DataFrame, optional
        Base dataset with random feature
    LR_Yeo_random : pandas.DataFrame, optional
        Yeo-Johnson dataset with random feature
    model_metrics_dict : dict, optional
        Dictionary to store model metrics (will be updated with sector models)
    random_state : int, default=42
        Reproducibility seed
    test_size : float, default=0.2
        Proportion of data for testing
    """
    # Create dictionary to store metrics if none provided
    if model_metrics_dict is None:
        model_metrics_dict = {}
    
    # Check if random features are provided
    use_random_features = (LR_Base_random is not None and LR_Yeo_random is not None)
    
    # Identify sector columns
    sector_columns = [col for col in feature_df.columns if col.startswith('gics_sector_')]
    
    # Define minimum companies needed per sector for modeling
    MIN_COMPANIES = 50
    
    print("\n" + "="*65)
    print("Running Linear Regression Models by Sector")
    print("="*65)
    
    # For each sector, create and evaluate models
    for sector_col in sector_columns:
        sector_name = sector_col.replace('gics_sector_', '')
        # Filter for companies in this sector
        sector_mask = feature_df[sector_col] == 1
        X_sector = feature_df[sector_mask]
        y_sector = score_df[sector_mask]
        
        # Check if we have enough companies in this sector
        if len(X_sector) < MIN_COMPANIES:
            print(f"\nSkipping {sector_name} - insufficient data ({len(X_sector)} companies, need {MIN_COMPANIES})")
            continue
        
        print(f"\n{sector_name} Sector - {len(X_sector)} companies")
        print("-" * 50)
        
        # Create feature sets for this sector
        X_sector_base = X_sector[base_columns]
        X_sector_yeo = X_sector[yeo_columns]
        
        # For random features, we need to filter the random datasets by the same sector indices
        if use_random_features:
            # Get the indices of companies in this sector
            sector_indices = X_sector.index
            
            # Filter random datasets to only include companies in this sector
            X_sector_base_random = LR_Base_random.loc[sector_indices]
            X_sector_yeo_random = LR_Yeo_random.loc[sector_indices]
        
        # Simple train-test split for regular features
        X_train_base, X_test_base, y_train, y_test = train_test_split(
            X_sector_base, y_sector, test_size=test_size, random_state=random_state
        )
        
        # Use the same indices for the Yeo features
        train_indices = X_train_base.index
        test_indices = X_test_base.index
        X_train_yeo = X_sector_yeo.loc[train_indices]
        X_test_yeo = X_sector_yeo.loc[test_indices]
        
        # If using random features, split them too using the same indices
        if use_random_features:
            X_train_base_random = X_sector_base_random.loc[train_indices]
            X_test_base_random = X_sector_base_random.loc[test_indices]
            X_train_yeo_random = X_sector_yeo_random.loc[train_indices]
            X_test_yeo_random = X_sector_yeo_random.loc[test_indices]
        
        # Train and evaluate all model variations
        model_configs = [
            {'name': f"Sector_{sector_name}_Base", 'X_train': X_train_base, 'X_test': X_test_base, 'type': 'Base'}
        ]
        
        # Add Yeo model configuration
        model_configs.append(
            {'name': f"Sector_{sector_name}_Yeo", 'X_train': X_train_yeo, 'X_test': X_test_yeo, 'type': 'Yeo'}
        )
        
        # Add random feature model configurations if available
        if use_random_features:
            model_configs.append(
                {'name': f"Sector_{sector_name}_Base_Random", 'X_train': X_train_base_random, 'X_test': X_test_base_random, 'type': 'Base+Random'}
            )
            model_configs.append(
                {'name': f"Sector_{sector_name}_Yeo_Random", 'X_train': X_train_yeo_random, 'X_test': X_test_yeo_random, 'type': 'Yeo+Random'}
            )
        
        # Train and evaluate all models
        for config in model_configs:
            # Train model
            model = LinearRegression()
            model.fit(config['X_train'], y_train)
            y_pred = model.predict(config['X_test'])
            
            # Calculate metrics
            mse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            rmse = np.sqrt(mse)
            
            # Store results
            model_metrics_dict[config['name']] = {
                'RMSE': rmse,
                'MAE': mae,
                'MSE': mse,
                'R2': r2,
                'n_companies': len(X_sector),  # Total number of companies in this sector
                'n_companies_train': len(X_train_base),  # Companies in training set
                'n_companies_test': len(X_test_base),  # Companies in test set
                'model': model,
                'y_test': y_test,
                'y_pred': y_pred,
                'sector': sector_name,
                'type': config['type']
            }
            
            # Print results
            print(f"  {config['type']} Model - RMSE: {rmse:.4f}, MAE: {mae:.4f}, MSE: {mse:.4f}, R²: {r2:.4f}")
    
    # Print summary of all sector models
    print("\n" + "="*75)
    print("Summary of Sector-Specific Model Performance")
    print("="*75)
    print(f"{'Sector':<20} {'Model Type':<15} {'RMSE':<8} {'MAE':<8} {'MSE':<8} {'R²':<8} {'n':<6}")
    print("-" * 75)
    
    # Filter for only sector models
    sector_models = {k: v for k, v in model_metrics_dict.items() if k.startswith('Sector_')}
    
    # Print all sector models sorted by sector then type
    for sector in sorted(set(v['sector'] for v in sector_models.values())):
        sector_specific_models = {k: v for k, v in sector_models.items() if v['sector'] == sector}
        for model_name, metrics in sorted(sector_specific_models.items(), 
                                         key=lambda x: ('Random' in x[0], x[0])):
            print(f"{sector:<20} {metrics['type']:<15} {metrics['RMSE']:.4f}  {metrics['MAE']:.4f}  {metrics['MSE']:.4f}  {metrics['R2']:.4f}  {metrics['n_companies']:<6}")
    
    # Calculate and print averages by model type
    print("\n" + "="*75)
    print("Average Performance by Model Type")
    print("="*75)
    
    model_types = set(v['type'] for v in sector_models.values())
    
    for model_type in sorted(model_types):
        models = {k: v for k, v in sector_models.items() if v['type'] == model_type}
        if models:
            avg_rmse = np.mean([m['RMSE'] for m in models.values()])
            avg_mae = np.mean([m['MAE'] for m in models.values()])
            avg_mse = np.mean([m['MSE'] for m in models.values()])
            avg_r2 = np.mean([m['R2'] for m in models.values()])
            print(f"{model_type:<15} RMSE: {avg_rmse:.4f}, MAE: {avg_mae:.4f}, MSE: {avg_mse:.4f}, R²: {avg_r2:.4f}")
    
    return model_metrics_dict

## Model execution

In [32]:
%%time

# Initialize a dictionary to store all model metrics
model_metrics = {}

# Run All Models and Compare Performance
print("\n" + "="*50)
print("Running Linear Regression on All Feature Sets")
print("="*50)

# Define all datasets to use
dataset_configs = [
    {'data': LR_Base, 'name': 'LR_Base'},
    {'data': LR_Yeo, 'name': 'LR_Yeo'},
    {'data': LR_Base_random, 'name': 'LR_Base_Random'},
    {'data': LR_Yeo_random, 'name': 'LR_Yeo_Random'}
]

# Run regression on each dataset
for config in dataset_configs:
    print(f"\nProcessing dataset: {config['name']}")
    run_regression_model(config['data'], y, model_name=config['name'])

# Now run the sector-specific models with random features
run_sector_models(
    feature_df, 
    score_df, 
    base_columns, 
    yeo_columns,
    LR_Base_random=LR_Base_random,  # Pass the entire random datasets
    LR_Yeo_random=LR_Yeo_random,
    model_metrics_dict=model_metrics
)

# Print overall comparison
print("\n" + "="*65)
print("Overall Best Models Comparison")
print("="*65)

# Group models by type
global_models = {k: v for k, v in model_metrics.items() if not k.startswith('Sector_')}
sector_models = {k: v for k, v in model_metrics.items() if k.startswith('Sector_')}

# Find best models
best_global = min(global_models.items(), key=lambda x: x[1]['RMSE']) if global_models else (None, {})
best_sector = min(sector_models.items(), key=lambda x: x[1]['RMSE']) if sector_models else (None, {})

print("Best Models:")
if best_global[0]:
    print(f"  Best Global: {best_global[0]} (RMSE: {best_global[1]['RMSE']:.4f}, R²: {best_global[1]['R2']:.4f})")
if best_sector[0]:
    print(f"  Best Sector: {best_sector[0]} (RMSE: {best_sector[1]['RMSE']:.4f}, R²: {best_sector[1]['R2']:.4f})")
    
if best_global[0] and best_sector[0]:
    print(f"  Improvement: RMSE {(best_global[1]['RMSE'] - best_sector[1]['RMSE']):.4f}, R² {(best_sector[1]['R2'] - best_global[1]['R2']):.4f}")


Running Linear Regression on All Feature Sets

Processing dataset: LR_Base
Sector distribution check:
Consumer Discretionary: Train 12.2%, Test 11.1%
Consumer Staples: Train 7.2%, Test 7.0%
Energy: Train 4.4%, Test 4.5%
Financials: Train 16.0%, Test 16.1%
Health Care: Train 8.9%, Test 8.8%
Industrials: Train 20.0%, Test 20.0%
Information Technology: Train 10.3%, Test 10.2%
Materials: Train 8.6%, Test 8.6%
Real Estate: Train 2.4%, Test 2.3%
Utilities: Train 4.7%, Test 4.8%

Model: LR_Base
  RMSE: 1.8645
  MAE : 1.4509
  MSE : 3.4763
  R²  : 0.1066

Processing dataset: LR_Yeo
Sector distribution check:
Consumer Discretionary: Train 12.2%, Test 11.1%
Consumer Staples: Train 7.2%, Test 7.0%
Energy: Train 4.4%, Test 4.5%
Financials: Train 16.0%, Test 16.1%
Health Care: Train 8.9%, Test 8.8%
Industrials: Train 20.0%, Test 20.0%
Information Technology: Train 10.3%, Test 10.2%
Materials: Train 8.6%, Test 8.6%
Real Estate: Train 2.4%, Test 2.3%
Utilities: Train 4.7%, Test 4.8%

Model: LR_Yeo
 

In [33]:
with open('strat_linreg_eval_metrics/model_results.pkl', 'wb') as f:
    pickle.dump(model_metrics, f)

In [34]:

results_df = pd.DataFrame([
    {
        'model_name': name,
        'RMSE': metrics['RMSE'],
        'MAE': metrics['MAE'],
        'MSE': metrics['MSE'],
        'R2': metrics['R2'],
        'is_sector_model': name.startswith('Sector_'),
        'sector': metrics.get('sector', 'Global'),
        'type': metrics.get('type', 'Global'),
        'n_companies': metrics.get('n_companies', 0),  # Add number of companies
        'n_companies_train': metrics.get('n_companies_train', 0),  # Training set size
        'n_companies_test': metrics.get('n_companies_test', 0)  # Test set size
    }
    for name, metrics in model_metrics.items()
])

# Save to CSV
results_df.to_csv('strat_linreg_eval_metrics/model_comparison_results.csv', index=False)

# 4 Elastic net tuning

In [80]:
%%time

# Suppress all warnings
warnings.filterwarnings('ignore')

print("\n" + "="*50)
print("Finding Optimal ElasticNet Parameters for All Datasets with Stratified Splitting")
print("="*50)

# Hyperparameter grid
lamb = 10 ** np.linspace(-1, 0.2, 15)  # Regularization strengths (alpha)
ratio = np.linspace(0, 1, 10)          # L1/L2 mixing (l1_ratio)

# All datasets and target
datasets = [LR_Base, LR_Yeo, LR_Base_random, LR_Yeo_random]
dataset_names = ['LR_Base', 'LR_Yeo', 'LR_Base_random', 'LR_Yeo_random']
target = score_df

# Storage for results
results_summary = []

# Iterate through datasets
for idx, X in enumerate(datasets):
    print(f"\nProcessing dataset: {dataset_names[idx]}...")
    
    # Extract sector information for stratification
    sector_columns = [col for col in X.columns if col.startswith('gics_sector_')]
    sector_data = X[sector_columns].copy()
    sector_labels = np.zeros(len(X), dtype=int)
    
    for i, col in enumerate(sector_columns):
        sector_labels[sector_data[col] == 1] = i
    
    # Split into train/test using stratification
    X_train, X_test, y_train, y_test, sector_train, sector_test = train_test_split(
        X, target, sector_labels, test_size=0.2, random_state=42, stratify=sector_labels
    )

    # Verify sector distribution
    print("  Sector distribution in train/test sets:")
    for i, col in enumerate(sector_columns):
        sector_name = col.replace('gics_sector_', '')
        train_pct = np.mean(sector_train == i) * 100
        test_pct = np.mean(sector_test == i) * 100
        print(f"    {sector_name}: Train {train_pct:.1f}%, Test {test_pct:.1f}%")
    
    # Create stratified k-fold for cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # Grid search with stratified cross-validation
    get_results = []
    
    for la in lamb:
        for r in ratio:
            # Wrap in try-except to handle any issues
            try:
                # Initialize scores collection
                rmse_folds = []
                
                # Manually perform stratified cross-validation
                for train_idx, val_idx in skf.split(X_train, sector_train):
                    # Create train/validation splits while preserving sector proportions
                    X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                    y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
                    
                    # Train model
                    model = ElasticNet(alpha=la, l1_ratio=r, random_state=42, max_iter=10000)
                    model.fit(X_fold_train, y_fold_train)
                    
                    # Predict and calculate RMSE
                    y_pred = model.predict(X_fold_val)
                    mse = mean_squared_error(y_fold_val, y_pred)
                    rmse = np.sqrt(mse)
                    rmse_folds.append(rmse)
                
                # Calculate mean RMSE across folds
                mean_rmse = np.mean(rmse_folds)
                get_results.append((la, r, mean_rmse, rmse_folds))

            except Exception as e:
                print(f"  Warning: Error with alpha={la}, l1_ratio={r}: {str(e)}")

    # Extract best results if we have any
    if get_results:
        # Find the result with minimum RMSE
        best_result = min(get_results, key=lambda x: x[2])  # x[2] is mean_rmse
        best_alpha, best_l1_ratio, least_error, best_rmse_folds = best_result

        # Save results
        results_summary.append({
            'dataset': dataset_names[idx],
            'least_error': least_error,
            'best_params': (best_alpha, best_l1_ratio),
            'rmse_folds': best_rmse_folds,
            'n_companies': len(X),
            'n_companies_train': len(X_train),
            'n_companies_test': len(X_test)
        })

        # Print intermediate results
        print(f'Dataset: {dataset_names[idx]}')
        print(f'  → Least CV RMSE: {least_error:.4f}')
        print(f'  → Best parameters (alpha, l1_ratio): {(best_alpha, best_l1_ratio)}')
        print(f'  → Number of companies: {len(X)} (train: {len(X_train)}, test: {len(X_test)})')

    else:
        print(f"  No valid results for {dataset_names[idx]}")

# Final summary
print("\n" + "="*50)
print("Summary of Best ElasticNet Parameters per Dataset")
print("="*50)

for res in results_summary:
    print(f"{res['dataset']}: RMSE = {res['least_error']:.4f}, Best Params = {res['best_params']}, Companies = {res['n_companies']}")

# Save results to file
with open('strat_linreg_eval_metrics/elasticnet_stratified_params.pkl', 'wb') as f:
    pickle.dump(results_summary, f)
print("\nElasticNet parameters saved to 'strat_linreg_eval_metrics/elasticnet_stratified_params.pkl'")


Finding Optimal ElasticNet Parameters for All Datasets with Stratified Splitting

Processing dataset: LR_Base...
  Sector distribution in train/test sets:
    Consumer Discretionary: Train 17.6%, Test 17.7%
    Consumer Staples: Train 7.2%, Test 7.0%
    Energy: Train 4.4%, Test 4.5%
    Financials: Train 16.0%, Test 16.1%
    Health Care: Train 8.9%, Test 8.8%
    Industrials: Train 20.0%, Test 20.0%
    Information Technology: Train 10.3%, Test 10.2%
    Materials: Train 8.6%, Test 8.6%
    Real Estate: Train 2.4%, Test 2.3%
    Utilities: Train 4.7%, Test 4.8%
Dataset: LR_Base
  → Least CV RMSE: 1.9162
  → Best parameters (alpha, l1_ratio): (np.float64(1.5848931924611136), np.float64(1.0))
  → Number of companies: 2202 (train: 1761, test: 441)

Processing dataset: LR_Yeo...
  Sector distribution in train/test sets:
    Consumer Discretionary: Train 17.6%, Test 17.7%
    Consumer Staples: Train 7.2%, Test 7.0%
    Energy: Train 4.4%, Test 4.5%
    Financials: Train 16.0%, Test 16.1%

## 4.1 run models with optimized elastic net parameters

In [35]:
# Run ElasticNet models with optimal parameters from search
print("\n" + "="*50)
print("Running ElasticNet Models with Optimal Parameters")
print("="*50)

# Load the optimal parameters from file
try:
    with open('strat_linreg_eval_metrics/elasticnet_stratified_params.pkl', 'rb') as f:
        elasticnet_results = pickle.load(f)
    
    # Convert results to a dictionary for easier access
    params_dict = {res['dataset']: res for res in elasticnet_results}
    
    print("Loaded optimal parameters:")
    for dataset, res in params_dict.items():
        print(f"  {dataset}: alpha={res['best_params'][0]:.6f}, l1_ratio={res['best_params'][1]:.6f}")
except FileNotFoundError:
    print("Parameter file not found. Run parameter search first.")
    # Define default parameters if file is not found
    params_dict = {
        'LR_Base': {'best_params': (0.1, 0.0)},
        'LR_Yeo': {'best_params': (0.1, 0.0)},
        'LR_Base_random': {'best_params': (0.1, 0.0)},
        'LR_Yeo_random': {'best_params': (0.1, 0.0)}
    }
# Save the consolidated model_metrics containing both linear regression and ElasticNet results
with open('strat_linreg_eval_metrics/model_results.pkl', 'wb') as f:
    pickle.dump(model_metrics, f)

# Define datasets to process
dataset_configs = [
    {'data': LR_Base, 'name': 'ElasticNet_LR_Base', 'param_key': 'LR_Base'},
    {'data': LR_Yeo, 'name': 'ElasticNet_LR_Yeo', 'param_key': 'LR_Yeo'},
    {'data': LR_Base_random, 'name': 'ElasticNet_LR_Base_Random', 'param_key': 'LR_Base_random'},
    {'data': LR_Yeo_random, 'name': 'ElasticNet_LR_Yeo_Random', 'param_key': 'LR_Yeo_random'}
]

# Function to run ElasticNet with stratified splitting
def run_elasticnet_model_stratified(X_data, y_data, model_name, param_key,
                                  random_state=42, test_size=0.2):
    """
    Run ElasticNet regression with stratified splitting.
    
    Parameters:
    -----------
    X_data : pandas.DataFrame
        Feature data
    y_data : pandas.Series or DataFrame
        Target variable
    model_name : str
        Identifier for the model
    param_key : str
        Key to lookup parameters in params_dict
    random_state : int, default=42
        Random seed
    test_size : float, default=0.2
        Test set proportion
    """
    # Get optimal parameters
    alpha, l1_ratio = params_dict[param_key]['best_params']
    
    # Use stratified split by sector
    X_train, X_test, y_train, y_test = perform_stratified_split_by_sector(
        X_data, y_data, test_size=test_size, random_state=random_state
    )

    # Initialize and train model
    model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=random_state, max_iter=10000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mse)

    # Log results
    print(f"\nModel: {model_name}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE : {mae:.4f}")
    print(f"  MSE : {mse:.4f}")
    print(f"  R²  : {r2:.4f}")
    print(f"  Parameters: alpha={alpha:.6f}, l1_ratio={l1_ratio:.6f}")

    # Count non-zero coefficients to assess feature selection
    n_features_used = np.sum(model.coef_ != 0)
    print(f"  Features used: {n_features_used} out of {len(X_data.columns)}")

    # Store metrics in the global dictionary
    model_metrics[model_name] = {
        'RMSE': rmse,
        'MAE': mae,
        'MSE': mse,
        'R2': r2,
        'n_companies': len(X_data),
        'n_companies_train': len(X_train),
        'n_companies_test': len(X_test),
        'model': model,
        'y_test': y_test,
        'y_pred': y_pred,
        'type': 'ElasticNet',
        'alpha': alpha,
        'l1_ratio': l1_ratio,
        'n_features_used': n_features_used
    }

# Run models
for config in dataset_configs:
    run_elasticnet_model_stratified(
        config['data'], 
        y, 
        config['name'], 
        config['param_key']  # Use the explicit param_key instead of derived one
    )

# Update results DataFrame with ElasticNet models
print("\n" + "="*50)
print("Updating Results DataFrame with ElasticNet Models")
print("="*50)

results_df = pd.DataFrame([
    {
        'model_name': name,
        'RMSE': metrics['RMSE'],
        'MAE': metrics['MAE'],
        'MSE': metrics['MSE'],
        'R2': metrics['R2'],
        'is_sector_model': name.startswith('Sector_'),
        'sector': metrics.get('sector', 'Global'),
        'type': metrics.get('type', 'Global'),
        'n_companies': metrics.get('n_companies', 0),
        'n_companies_train': metrics.get('n_companies_train', 0),
        'n_companies_test': metrics.get('n_companies_test', 0),
        'alpha': metrics.get('alpha', None),
        'l1_ratio': metrics.get('l1_ratio', None),
        'n_features_used': metrics.get('n_features_used', None)
    }
    for name, metrics in model_metrics.items()
])

# Save to CSV
results_df.to_csv('strat_linreg_eval_metrics/model_comparison_results_with_elasticnet.csv', index=False)
print("Results saved to 'strat_linreg_eval_metrics/model_comparison_results_with_elasticnet.csv'")

# Print summary of best models
print("\nTop 5 models by RMSE:")
top_models = results_df.sort_values('RMSE').head(5)
for _, row in top_models.iterrows():
    print(f"  {row['model_name']}: RMSE = {row['RMSE']:.4f}, R² = {row['R2']:.4f}, Type = {row['type']}")

# Save the updated model_metrics with ElasticNet models
with open('strat_linreg_eval_metrics/model_results.pkl', 'wb') as f:
    pickle.dump(model_metrics, f)
print("Updated model_metrics saved to 'strat_linreg_eval_metrics/model_results.pkl'")


Running ElasticNet Models with Optimal Parameters
Loaded optimal parameters:
  LR_Base: alpha=1.584893, l1_ratio=1.000000
  LR_Yeo: alpha=0.100000, l1_ratio=0.000000
  LR_Base_random: alpha=1.584893, l1_ratio=1.000000
  LR_Yeo_random: alpha=0.100000, l1_ratio=0.000000
Sector distribution check:
Consumer Discretionary: Train 12.2%, Test 11.1%
Consumer Staples: Train 7.2%, Test 7.0%
Energy: Train 4.4%, Test 4.5%
Financials: Train 16.0%, Test 16.1%
Health Care: Train 8.9%, Test 8.8%
Industrials: Train 20.0%, Test 20.0%
Information Technology: Train 10.3%, Test 10.2%
Materials: Train 8.6%, Test 8.6%
Real Estate: Train 2.4%, Test 2.3%
Utilities: Train 4.7%, Test 4.8%

Model: ElasticNet_LR_Base
  RMSE: 1.9610
  MAE : 1.5514
  MSE : 3.8453
  R²  : 0.0118
  Parameters: alpha=1.584893, l1_ratio=1.000000
  Features used: 14 out of 362
Sector distribution check:
Consumer Discretionary: Train 12.2%, Test 11.1%
Consumer Staples: Train 7.2%, Test 7.0%
Energy: Train 4.4%, Test 4.5%
Financials: Train

In [36]:
# For a model stored in model_metrics
model = model_metrics['ElasticNet_LR_Base']['model']
non_zero_features = [(feature, coef) for feature, coef in zip(LR_Base.columns, model.coef_) if coef != 0]
print(f"Features with non-zero coefficients ({len(non_zero_features)} out of {len(LR_Base.columns)}):")
for feature, coef in sorted(non_zero_features, key=lambda x: abs(x[1]), reverse=True):
    print(f"  {feature}: {coef:.6f}")

Features with non-zero coefficients (14 out of 362):
  hist_roe: 0.000892
  hist_fcf_yld: 0.000087
  hist_pe: -0.000058
  hist_eps_usd: 0.000013
  hist_ebitda_ev: 0.000005
  shares_float: 0.000000
  shares_outstanding: -0.000000
  hist_rd_exp_usd: -0.000000
  net_income_usd: -0.000000
  hist_gross_profit_usd: 0.000000
  market_cap_usd: 0.000000
  hist_ev_usd: -0.000000
  hist_net_debt_usd: 0.000000
  hist_net_chg_lt_debt_usd: -0.000000


In [37]:
print("Available models in model_metrics:")
for key in model_metrics.keys():
    print(f"- {key}")

Available models in model_metrics:
- LR_Base
- LR_Yeo
- LR_Base_Random
- LR_Yeo_Random
- Sector_Consumer Discretionary_Base
- Sector_Consumer Discretionary_Yeo
- Sector_Consumer Discretionary_Base_Random
- Sector_Consumer Discretionary_Yeo_Random
- Sector_Consumer Staples_Base
- Sector_Consumer Staples_Yeo
- Sector_Consumer Staples_Base_Random
- Sector_Consumer Staples_Yeo_Random
- Sector_Energy_Base
- Sector_Energy_Yeo
- Sector_Energy_Base_Random
- Sector_Energy_Yeo_Random
- Sector_Financials_Base
- Sector_Financials_Yeo
- Sector_Financials_Base_Random
- Sector_Financials_Yeo_Random
- Sector_Health Care_Base
- Sector_Health Care_Yeo
- Sector_Health Care_Base_Random
- Sector_Health Care_Yeo_Random
- Sector_Industrials_Base
- Sector_Industrials_Yeo
- Sector_Industrials_Base_Random
- Sector_Industrials_Yeo_Random
- Sector_Information Technology_Base
- Sector_Information Technology_Yeo
- Sector_Information Technology_Base_Random
- Sector_Information Technology_Yeo_Random
- Sector_Material

# Plot outcomes function

In [89]:
def perform_residual_analysis_fixed(model_metrics, model_names, output_dir=None):
    """
    Perform comprehensive residual analysis on selected models.
    
    Parameters:
    -----------
    model_metrics : dict
        Dictionary containing model metrics and predictions
    model_names : list
        List of model names to analyze
    output_dir : str, optional
        Directory to save plots. If None, plots are not saved.
    
    Returns:
    --------
    pandas.DataFrame
        Summary statistics of residual analysis
    """
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import scipy.stats as stats
    import seaborn as sns
    from sklearn.metrics import mean_squared_error, r2_score
    import os
    import pickle
    
    # Create output directory if specified and doesn't exist
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
    
    # Create summary statistics dataframe
    residual_stats = []
    
    # Analysis for each model - Individual plots instead of combined
    for model_name in model_names:
        # Extract predictions and actual values
        metrics = model_metrics[model_name]
        y_test = metrics['y_test'].values.flatten()
        y_pred = metrics['y_pred'].flatten()
        
        # Calculate residuals
        residuals = y_test - y_pred
        
        # Basic stats
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        mean_residual = np.mean(residuals)
        std_residual = np.std(residuals)
        
        # Test for normality (Shapiro-Wilk)
        if len(residuals) <= 5000:  # Shapiro-Wilk limited to 5000 samples
            shapiro_stat, shapiro_p = stats.shapiro(residuals)
        else:
            shapiro_stat, shapiro_p = np.nan, np.nan
            
        # Test for heteroscedasticity (Breusch-Pagan)
        # First, fit a model to predict squared residuals from predicted values
        bp_model = np.polyfit(y_pred, residuals**2, 1)
        bp_fit = np.polyval(bp_model, y_pred)
        bp_stat, bp_p = stats.pearsonr(y_pred, residuals**2)
        
        # Store summary statistics
        residual_stats.append({
            'Model': model_name,
            'RMSE': rmse,
            'R²': r2,
            'Mean Residual': mean_residual,
            'Std Residual': std_residual,
            'Shapiro-Wilk p': shapiro_p,
            'BP Test p': bp_p,
            'Residual Range': [np.min(residuals), np.max(residuals)],
            'Residual IQR': np.subtract(*np.percentile(residuals, [75, 25]))
        })
        
        # Create individual figure for each model with horizontal layout
        if output_dir:
            fig, axes = plt.subplots(1, 4, figsize=(20, 5))
            plt.subplots_adjust(wspace=0.3)
            
            # 1. Residuals vs Predicted plot
            ax = axes[0]
            ax.scatter(y_pred, residuals, alpha=0.5, s=10)
            ax.axhline(y=0, color='r', linestyle='-', alpha=0.3)
            
            # Add LOESS smoothing
            try:
                from statsmodels.nonparametric.smoothers_lowess import lowess
                z = lowess(residuals, y_pred, frac=0.3)
                ax.plot(z[:, 0], z[:, 1], 'r-', linewidth=2)
            except:
                # If LOWESS fails, use simple polynomial fit
                p = np.polyfit(y_pred, residuals, 2)
                ax.plot(np.sort(y_pred), np.polyval(p, np.sort(y_pred)), 'r-', linewidth=2)
                
            ax.set_xlabel('Predicted Values')
            ax.set_ylabel('Residuals')
            ax.set_title('Residuals vs Predicted')
            ax.grid(True, alpha=0.3)
            
            # Add model metrics to plot
            textstr = f'RMSE: {rmse:.4f}\nR²: {r2:.4f}\nBP Test p: {bp_p:.6f}'
            props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
            ax.annotate(textstr, xy=(0.05, 0.95), xycoords='axes fraction',
                        fontsize=9, verticalalignment='top', bbox=props)
            
            # 2. Q-Q plot (check for normality) - FIXED VERSION
            ax = axes[1]
            temp_fig = plt.figure()  # Create a temporary figure
            stats.probplot(residuals, dist="norm", plot=plt)
            # Get the lines and points from the current figure
            lines = plt.gca().get_lines()
            ax.plot(lines[0].get_xdata(), lines[0].get_ydata(), 'o', markersize=5)
            ax.plot(lines[1].get_xdata(), lines[1].get_ydata(), 'r-')
            plt.close(temp_fig)  # Close the temporary figure
            ax.set_title(f'Q-Q Plot (Shapiro-Wilk p: {shapiro_p:.6f})')
            
            # 3. Scale-Location Plot (sqrt(|standardized residuals|) vs predicted)
            ax = axes[2]
            standardized_residuals = residuals / std_residual
            ax.scatter(y_pred, np.sqrt(np.abs(standardized_residuals)), alpha=0.5, s=10)
            
            # Add LOESS or polynomial smoothing
            try:
                z = lowess(np.sqrt(np.abs(standardized_residuals)), y_pred, frac=0.3)
                ax.plot(z[:, 0], z[:, 1], 'r-', linewidth=2)
            except:
                p = np.polyfit(y_pred, np.sqrt(np.abs(standardized_residuals)), 2)
                ax.plot(np.sort(y_pred), np.polyval(p, np.sort(y_pred)), 'r-', linewidth=2)
                
            ax.set_xlabel('Predicted Values')
            ax.set_ylabel('√|Standardized Residuals|')
            ax.set_title('Scale-Location Plot')
            ax.grid(True, alpha=0.3)
            
            # 4. Residual Histogram
            ax = axes[3]
            ax.hist(residuals, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
            ax.set_xlabel('Residual Value')
            ax.set_ylabel('Frequency')
            ax.set_title('Residual Distribution')
            ax.grid(True, alpha=0.3)
            
            # Add normal curve overlay
            xmin, xmax = ax.get_xlim()
            x = np.linspace(xmin, xmax, 100)
            p = stats.norm.pdf(x, mean_residual, std_residual)
            p = p * (len(residuals) * (xmax - xmin) / 30) # Scale to match histogram height
            ax.plot(x, p, 'k', linewidth=2)
            
            # Add metrics
            textstr = f'Mean: {mean_residual:.4f}\nStd: {std_residual:.4f}'
            ax.annotate(textstr, xy=(0.05, 0.95), xycoords='axes fraction',
                        fontsize=9, verticalalignment='top', bbox=props)
            
            # Add overall title
            fig.suptitle(f'Residual Analysis: {model_name}', fontsize=16)
            plt.tight_layout(rect=[0, 0, 1, 0.95])
            
            # Save figure
            plt.savefig(f'{output_dir}/{model_name}_residual_analysis.png', dpi=300)
            plt.close()
            
            # Create Actual vs Predicted plot
            plt.figure(figsize=(10, 8))
            plt.scatter(y_test, y_pred, alpha=0.5)
            
            # Add perfect prediction line
            min_val = min(np.min(y_test), np.min(y_pred))
            max_val = max(np.max(y_test), np.max(y_pred))
            plt.plot([min_val, max_val], [min_val, max_val], 'r--')
            
            plt.xlabel('Actual Values')
            plt.ylabel('Predicted Values')
            plt.title(f'{model_name} - Actual vs Predicted')
            plt.grid(True, alpha=0.3)
            
            # Add metrics to plot
            textstr = f'RMSE: {rmse:.4f}\nR²: {r2:.4f}'
            props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
            plt.annotate(textstr, xy=(0.05, 0.95), xycoords='axes fraction',
                        fontsize=12, verticalalignment='top', bbox=props)
            
            plt.tight_layout()
            plt.savefig(f'{output_dir}/{model_name}_actual_vs_predicted.png', dpi=300)
            plt.close()
            
            # Residuals by ESG Score bins
            plt.figure(figsize=(12, 8))
            
            # Create ESG score bins for analysis
            score_bins = np.linspace(np.min(y_test), np.max(y_test), 10)
            bin_indices = np.digitize(y_test, score_bins)
            
            # Calculate mean residual and std for each bin
            bin_residuals = [residuals[bin_indices == i] for i in range(1, len(score_bins)+1)]
            bin_means = [np.mean(res) if len(res) > 0 else np.nan for res in bin_residuals]
            bin_stds = [np.std(res) if len(res) > 0 else np.nan for res in bin_residuals]
            bin_centers = [(score_bins[i-1] + score_bins[i])/2 if i < len(score_bins) else score_bins[-1] 
                        for i in range(1, len(score_bins)+1)]
            
            # Plot
            plt.errorbar(bin_centers, bin_means, yerr=bin_stds, fmt='o-', capsize=5)
            plt.axhline(y=0, color='r', linestyle='--')
            plt.xlabel('ESG Score Bin')
            plt.ylabel('Mean Residual ± Std Dev')
            plt.title(f'{model_name} - Residuals by ESG Score')
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.savefig(f'{output_dir}/{model_name}_residuals_by_score.png', dpi=300)
            plt.close()
    
    # Create R-squared and RMSE comparison plot
    if output_dir and len(model_names) > 1:
        plt.figure(figsize=(12, 8))
        r2_values = [model_metrics[name]['R2'] for name in model_names]
        rmse_values = [model_metrics[name]['RMSE'] for name in model_names]
        
        plt.subplot(1, 2, 1)
        bars = plt.bar(model_names, r2_values)
        plt.title('R² Comparison')
        plt.ylabel('R²')
        plt.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        
        # Add values on bars
        for bar, val in zip(bars, r2_values):
            plt.text(bar.get_x() + bar.get_width()/2, val + 0.01, f'{val:.4f}', 
                    ha='center', va='bottom', rotation=0)
        
        plt.subplot(1, 2, 2)
        bars = plt.bar(model_names, rmse_values)
        plt.title('RMSE Comparison')
        plt.ylabel('RMSE')
        plt.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        
        # Add values on bars
        for bar, val in zip(bars, rmse_values):
            plt.text(bar.get_x() + bar.get_width()/2, val + 0.05, f'{val:.4f}', 
                    ha='center', va='bottom', rotation=0)
        
        plt.tight_layout()
        plt.savefig(f'{output_dir}/model_performance_comparison.png', dpi=300)
        plt.close()
    
    # NEW CODE: Add boxplot visualization for RMSE distribution
    if output_dir:
        # Helper function for calculating confidence intervals
        def mean_confidence_interval(data, confidence=0.95):
            a = 1.0 * np.array(data)
            n = len(a)
            m, se = np.mean(a), stats.sem(a)
            h = se * stats.t.ppf((1 + confidence) / 2., n-1)
            return m, m-h, m+h
        
        # Try to load elasticnet parameters to extract RMSE folds
        try:
            with open(f'{output_dir}/../elasticnet_stratified_params.pkl', 'rb') as f:
                elasticnet_results = pickle.load(f)
            
            # Create a DataFrame for the RMSE values from cross-validation
            rmse_data = []
            for result in elasticnet_results:
                if 'rmse_folds' in result and isinstance(result['rmse_folds'], list):
                    dataset = result['dataset']
                    for fold_idx, rmse_val in enumerate(result['rmse_folds']):
                        rmse_data.append({
                            'Dataset': dataset,
                            'Fold': fold_idx + 1,
                            'RMSE': rmse_val
                        })
            
            if rmse_data:
                rmse_df = pd.DataFrame(rmse_data)
                
                plt.figure(figsize=(10, 6))
                
                # Boxplot for RMSE distribution
                box = sns.boxplot(x='Dataset', y='RMSE', data=rmse_df, palette='pastel')
                
                # Stripplot for individual fold RMSEs
                strip = sns.stripplot(x='Dataset', y='RMSE', data=rmse_df, color='gray', alpha=0.6, jitter=True)
                
                # Plot mean and 95% CI as red points with error bars
                for i, dataset in enumerate(rmse_df['Dataset'].unique()):
                    rmse_vals = rmse_df[rmse_df['Dataset'] == dataset]['RMSE']
                    mean = np.mean(rmse_vals)
                    ci_low, ci_high = mean_confidence_interval(rmse_vals)[1:]
                    err = plt.errorbar(i, mean, yerr=[[mean - ci_low], [ci_high - mean]],
                                    fmt='o', color='red', capsize=5, label='Mean ± 95% CI' if i == 0 else "")
                
                # Title and axes
                plt.title('ElasticNet RMSE Distribution per Dataset')
                plt.ylabel('RMSE (lower is better)')
                plt.xlabel('Dataset')
                plt.xticks(rotation=15)
                plt.grid(axis='y', linestyle='--', alpha=0.5)
                
                # Custom legend
                from matplotlib.lines import Line2D
                legend_elements = [
                    Line2D([0], [0], marker='s', color='w', label='RMSE Distribution (Boxplot)',
                        markerfacecolor='lightblue', markersize=15),
                    Line2D([0], [0], marker='o', color='gray', label='Individual CV Fold RMSE',
                        linestyle='None', markersize=8, alpha=0.6),
                    Line2D([0], [0], marker='o', color='red', label='Mean ± 95% CI',
                        linestyle='None', markersize=8)
                ]
                plt.legend(handles=legend_elements, loc='upper right')
                
                plt.tight_layout()
                plt.savefig(f'{output_dir}/elasticnet_rmse_distribution.png', dpi=300)
                plt.close()
        except Exception as e:
            print(f"Could not create RMSE distribution plot: {str(e)}")
    
    # Create a summary DataFrame
    summary_df = pd.DataFrame(residual_stats)
    
    # Save summary to CSV if output directory specified
    if output_dir:
        summary_df.to_csv(f'{output_dir}/residual_analysis_summary.csv', index=False)
    
    return summary_df

In [ ]:
# def perform_residual_analysis_fixed(model_metrics, model_names, output_dir=None):
#     """
#     Perform comprehensive residual analysis on selected models.
    
#     Parameters:
#     -----------
#     model_metrics : dict
#         Dictionary containing model metrics and predictions
#     model_names : list
#         List of model names to analyze
#     output_dir : str, optional
#         Directory to save plots. If None, plots are not saved.
    
#     Returns:
#     --------
#     pandas.DataFrame
#         Summary statistics of residual analysis
#     """
#     import matplotlib.pyplot as plt
#     import numpy as np
#     import pandas as pd
#     import scipy.stats as stats
#     from sklearn.metrics import mean_squared_error, r2_score
#     import os
    
#     # Create output directory if specified and doesn't exist
#     if output_dir:
#         os.makedirs(output_dir, exist_ok=True)
    
#     # Create summary statistics dataframe
#     residual_stats = []
    
#     # Analysis for each model - Individual plots instead of combined
#     for model_name in model_names:
#         # Extract predictions and actual values
#         metrics = model_metrics[model_name]
#         y_test = metrics['y_test'].values.flatten()
#         y_pred = metrics['y_pred'].flatten()
        
#         # Calculate residuals
#         residuals = y_test - y_pred
        
#         # Basic stats
#         rmse = np.sqrt(mean_squared_error(y_test, y_pred))
#         r2 = r2_score(y_test, y_pred)
#         mean_residual = np.mean(residuals)
#         std_residual = np.std(residuals)
        
#         # Test for normality (Shapiro-Wilk)
#         if len(residuals) <= 5000:  # Shapiro-Wilk limited to 5000 samples
#             shapiro_stat, shapiro_p = stats.shapiro(residuals)
#         else:
#             shapiro_stat, shapiro_p = np.nan, np.nan
            
#         # Test for heteroscedasticity (Breusch-Pagan)
#         # First, fit a model to predict squared residuals from predicted values
#         bp_model = np.polyfit(y_pred, residuals**2, 1)
#         bp_fit = np.polyval(bp_model, y_pred)
#         bp_stat, bp_p = stats.pearsonr(y_pred, residuals**2)
        
#         # Store summary statistics
#         residual_stats.append({
#             'Model': model_name,
#             'RMSE': rmse,
#             'R²': r2,
#             'Mean Residual': mean_residual,
#             'Std Residual': std_residual,
#             'Shapiro-Wilk p': shapiro_p,
#             'BP Test p': bp_p,
#             'Residual Range': [np.min(residuals), np.max(residuals)],
#             'Residual IQR': np.subtract(*np.percentile(residuals, [75, 25]))
#         })
        
#         # Create individual figure for each model with horizontal layout
#         if output_dir:
#             fig, axes = plt.subplots(1, 4, figsize=(20, 5))
#             plt.subplots_adjust(wspace=0.3)
            
#             # 1. Residuals vs Predicted plot
#             ax = axes[0]
#             ax.scatter(y_pred, residuals, alpha=0.5, s=10)
#             ax.axhline(y=0, color='r', linestyle='-', alpha=0.3)
            
#             # Add LOESS smoothing
#             try:
#                 from statsmodels.nonparametric.smoothers_lowess import lowess
#                 z = lowess(residuals, y_pred, frac=0.3)
#                 ax.plot(z[:, 0], z[:, 1], 'r-', linewidth=2)
#             except:
#                 # If LOWESS fails, use simple polynomial fit
#                 p = np.polyfit(y_pred, residuals, 2)
#                 ax.plot(np.sort(y_pred), np.polyval(p, np.sort(y_pred)), 'r-', linewidth=2)
                
#             ax.set_xlabel('Predicted Values')
#             ax.set_ylabel('Residuals')
#             ax.set_title('Residuals vs Predicted')
#             ax.grid(True, alpha=0.3)
            
#             # Add model metrics to plot
#             textstr = f'RMSE: {rmse:.4f}\nR²: {r2:.4f}\nBP Test p: {bp_p:.6f}'
#             props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
#             ax.annotate(textstr, xy=(0.05, 0.95), xycoords='axes fraction',
#                         fontsize=9, verticalalignment='top', bbox=props)
            
#             # 2. Q-Q plot (check for normality) - FIXED VERSION
#             ax = axes[1]
#             temp_fig = plt.figure()  # Create a temporary figure
#             stats.probplot(residuals, dist="norm", plot=plt)
#             # Get the lines and points from the current figure
#             lines = plt.gca().get_lines()
#             ax.plot(lines[0].get_xdata(), lines[0].get_ydata(), 'o', markersize=5)
#             ax.plot(lines[1].get_xdata(), lines[1].get_ydata(), 'r-')
#             plt.close(temp_fig)  # Close the temporary figure
#             ax.set_title(f'Q-Q Plot (Shapiro-Wilk p: {shapiro_p:.6f})')
            
#             # 3. Scale-Location Plot (sqrt(|standardized residuals|) vs predicted)
#             ax = axes[2]
#             standardized_residuals = residuals / std_residual
#             ax.scatter(y_pred, np.sqrt(np.abs(standardized_residuals)), alpha=0.5, s=10)
            
#             # Add LOESS or polynomial smoothing
#             try:
#                 z = lowess(np.sqrt(np.abs(standardized_residuals)), y_pred, frac=0.3)
#                 ax.plot(z[:, 0], z[:, 1], 'r-', linewidth=2)
#             except:
#                 p = np.polyfit(y_pred, np.sqrt(np.abs(standardized_residuals)), 2)
#                 ax.plot(np.sort(y_pred), np.polyval(p, np.sort(y_pred)), 'r-', linewidth=2)
                
#             ax.set_xlabel('Predicted Values')
#             ax.set_ylabel('√|Standardized Residuals|')
#             ax.set_title('Scale-Location Plot')
#             ax.grid(True, alpha=0.3)
            
#             # 4. Residual Histogram
#             ax = axes[3]
#             ax.hist(residuals, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
#             ax.set_xlabel('Residual Value')
#             ax.set_ylabel('Frequency')
#             ax.set_title('Residual Distribution')
#             ax.grid(True, alpha=0.3)
            
#             # Add normal curve overlay
#             xmin, xmax = ax.get_xlim()
#             x = np.linspace(xmin, xmax, 100)
#             p = stats.norm.pdf(x, mean_residual, std_residual)
#             p = p * (len(residuals) * (xmax - xmin) / 30) # Scale to match histogram height
#             ax.plot(x, p, 'k', linewidth=2)
            
#             # Add metrics
#             textstr = f'Mean: {mean_residual:.4f}\nStd: {std_residual:.4f}'
#             ax.annotate(textstr, xy=(0.05, 0.95), xycoords='axes fraction',
#                         fontsize=9, verticalalignment='top', bbox=props)
            
#             # Add overall title
#             fig.suptitle(f'Residual Analysis: {model_name}', fontsize=16)
#             plt.tight_layout(rect=[0, 0, 1, 0.95])
            
#             # Save figure
#             plt.savefig(f'{output_dir}/{model_name}_residual_analysis.png', dpi=300)
#             plt.close()
            
#             # Create Actual vs Predicted plot
#             plt.figure(figsize=(10, 8))
#             plt.scatter(y_test, y_pred, alpha=0.5)
            
#             # Add perfect prediction line
#             min_val = min(np.min(y_test), np.min(y_pred))
#             max_val = max(np.max(y_test), np.max(y_pred))
#             plt.plot([min_val, max_val], [min_val, max_val], 'r--')
            
#             plt.xlabel('Actual Values')
#             plt.ylabel('Predicted Values')
#             plt.title(f'{model_name} - Actual vs Predicted')
#             plt.grid(True, alpha=0.3)
            
#             # Add metrics to plot
#             textstr = f'RMSE: {rmse:.4f}\nR²: {r2:.4f}'
#             props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
#             plt.annotate(textstr, xy=(0.05, 0.95), xycoords='axes fraction',
#                         fontsize=12, verticalalignment='top', bbox=props)
            
#             plt.tight_layout()
#             plt.savefig(f'{output_dir}/{model_name}_actual_vs_predicted.png', dpi=300)
#             plt.close()
            
#             # Residuals by ESG Score bins
#             plt.figure(figsize=(12, 8))
            
#             # Create ESG score bins for analysis
#             score_bins = np.linspace(np.min(y_test), np.max(y_test), 10)
#             bin_indices = np.digitize(y_test, score_bins)
            
#             # Calculate mean residual and std for each bin
#             bin_residuals = [residuals[bin_indices == i] for i in range(1, len(score_bins)+1)]
#             bin_means = [np.mean(res) if len(res) > 0 else np.nan for res in bin_residuals]
#             bin_stds = [np.std(res) if len(res) > 0 else np.nan for res in bin_residuals]
#             bin_centers = [(score_bins[i-1] + score_bins[i])/2 if i < len(score_bins) else score_bins[-1] 
#                         for i in range(1, len(score_bins)+1)]
            
#             # Plot
#             plt.errorbar(bin_centers, bin_means, yerr=bin_stds, fmt='o-', capsize=5)
#             plt.axhline(y=0, color='r', linestyle='--')
#             plt.xlabel('ESG Score Bin')
#             plt.ylabel('Mean Residual ± Std Dev')
#             plt.title(f'{model_name} - Residuals by ESG Score')
#             plt.grid(True, alpha=0.3)
#             plt.tight_layout()
#             plt.savefig(f'{output_dir}/{model_name}_residuals_by_score.png', dpi=300)
#             plt.close()
    
#     # Create R-squared and RMSE comparison plot
#     if output_dir and len(model_names) > 1:
#         plt.figure(figsize=(12, 8))
#         r2_values = [model_metrics[name]['R2'] for name in model_names]
#         rmse_values = [model_metrics[name]['RMSE'] for name in model_names]
        
#         plt.subplot(1, 2, 1)
#         bars = plt.bar(model_names, r2_values)
#         plt.title('R² Comparison')
#         plt.ylabel('R²')
#         plt.grid(True, alpha=0.3)
#         plt.xticks(rotation=45)
        
#         # Add values on bars
#         for bar, val in zip(bars, r2_values):
#             plt.text(bar.get_x() + bar.get_width()/2, val + 0.01, f'{val:.4f}', 
#                     ha='center', va='bottom', rotation=0)
        
#         plt.subplot(1, 2, 2)
#         bars = plt.bar(model_names, rmse_values)
#         plt.title('RMSE Comparison')
#         plt.ylabel('RMSE')
#         plt.grid(True, alpha=0.3)
#         plt.xticks(rotation=45)
        
#         # Add values on bars
#         for bar, val in zip(bars, rmse_values):
#             plt.text(bar.get_x() + bar.get_width()/2, val + 0.05, f'{val:.4f}', 
#                     ha='center', va='bottom', rotation=0)
        
#         plt.tight_layout()
#         plt.savefig(f'{output_dir}/model_performance_comparison.png', dpi=300)
#         plt.close()
    
#     # Create a summary DataFrame
#     summary_df = pd.DataFrame(residual_stats)
    
#     # Save summary to CSV if output directory specified
#     if output_dir:
#         summary_df.to_csv(f'{output_dir}/residual_analysis_summary.csv', index=False)
    
#     return summary_df

In [22]:
# models to compare
models_to_compare = [
    'ElasticNet_LR_Yeo_Random',  # Best overall model
    'ElasticNet_LR_Base_Random',                    # Best linear regression model
    'ElasticNet_LR_Base',         # To isolate effect of ElasticNet on base features
    'ElasticNet_LR_Yeo'                      # To see effect of Yeo transform in linear model
]

# output directory
residual_plots_dir = 'strat_linreg_eval_metrics/residual_plots'

In [23]:
# Run plot function
summary_df = perform_residual_analysis_fixed(
    model_metrics=model_metrics,
    model_names=models_to_compare,
    output_dir=residual_plots_dir,
)
print(summary_df)

NameError: name 'perform_residual_analysis_fixed' is not defined

# 6 Implement Holm Bonferroni tests

In [88]:
import numpy as np
import pandas as pd
from scipy import stats

def perform_holm_bonferroni_tests(model_metrics, model_names, alpha=0.05):
    """
    Perform pairwise t-tests between models with Holm-Bonferroni correction.
    
    Parameters:
    -----------
    model_metrics : dict
        Dictionary containing model metrics, including y_pred and y_test for each model
    model_names : list
        List of model names to compare
    alpha : float, default=0.05
        Overall significance level
        
    Returns:
    --------
    pandas.DataFrame
        Results of pairwise t-tests with Holm-Bonferroni correction
    """
    results = []
    num_tests = len(model_names) * (len(model_names) - 1) // 2
    
    print(f"Running {num_tests} pairwise tests with Holm-Bonferroni correction")
    
    # Collect all test results
    for i, model_a in enumerate(model_names):
        for j, model_b in enumerate(model_names):
            if j <= i:  # Skip redundant comparisons
                continue
            
            # Get predictions and actual values
            y_pred_a = model_metrics[model_a]['y_pred']
            y_pred_b = model_metrics[model_b]['y_pred']
            y_test = model_metrics[model_a]['y_test']  # Same for both models
            
            # Convert to numpy arrays if they're DataFrames or Series
            if hasattr(y_pred_a, 'values'):
                y_pred_a = y_pred_a.values
            if hasattr(y_pred_b, 'values'):
                y_pred_b = y_pred_b.values
            if hasattr(y_test, 'values'):
                y_test = y_test.values
                
            # Ensure they're all flattened 1D arrays
            y_pred_a = y_pred_a.flatten()
            y_pred_b = y_pred_b.flatten()
            y_test = y_test.flatten()
            
            # Calculate squared errors
            squared_errors_a = np.square(y_pred_a - y_test)
            squared_errors_b = np.square(y_pred_b - y_test)
            
            # Perform paired t-test
            t_stat, p_value = stats.ttest_rel(squared_errors_a, squared_errors_b)
            
            # Store result
            results.append({
                'Model A': model_a,
                'Model B': model_b,
                't-statistic': t_stat,
                'p-value': p_value,
                'RMSE A': np.sqrt(np.mean(squared_errors_a)),
                'RMSE B': np.sqrt(np.mean(squared_errors_b))
            })
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    
    # Sort by p-value (ascending)
    results_df = results_df.sort_values('p-value')
    
    # Apply Holm-Bonferroni correction
    results_df['rank'] = range(1, len(results_df) + 1)
    results_df['adjusted threshold'] = alpha / (num_tests + 1 - results_df['rank'])
    results_df['significant'] = results_df['p-value'] < results_df['adjusted threshold']
    
    # Find the first non-significant test
    try:
        first_nonsig_idx = results_df[~results_df['significant']].index[0]
        # Mark all tests with higher p-values as non-significant
        results_df.loc[results_df.index >= first_nonsig_idx, 'significant'] = False
    except IndexError:
        # All tests are significant
        pass
    
    return results_df

# Example usage:
if __name__ == "__main__":
    # Load model metrics from pickle file
    with open('strat_linreg_eval_metrics/model_results.pkl', 'rb') as f:
        model_metrics = pickle.load(f)
    
    # Define models to compare
    models_to_compare = [
        'LR_Base', 
        'LR_Yeo', 
        'LR_Base_Random', 
        'LR_Yeo_Random',
        'ElasticNet_LR_Yeo_Random',  # Best overall model
        'ElasticNet_LR_Base_Random', # Best linear regression model
        'ElasticNet_LR_Base',        # To isolate effect of ElasticNet on base features
        'ElasticNet_LR_Yeo'          # To see effect of Yeo transform in linear model
    ]
    
    # Run pairwise t-tests with Holm-Bonferroni correction
    results = perform_holm_bonferroni_tests(model_metrics, models_to_compare)
    print(results)
    
    # Generate LaTeX table
    print("\nLaTeX Table:")
    print("\\begin{table}[H]")
    print("\\centering")
    print("\\caption{Pairwise t-tests Between Models with Holm-Bonferroni Correction}")
    print("\\label{tab:model-significance-holm}")
    print("\\begin{tabularx}{\\textwidth}{l l c c c c c c}")
    print("\\toprule")
    print("\\textbf{Model A} & \\textbf{Model B} & \\textbf{RMSE A} & \\textbf{RMSE B} & \\textbf{t-statistic} & \\textbf{p-value} & \\textbf{Adj. Threshold} & \\textbf{Sig.} \\\\")
    print("\\midrule")
    
    for _, row in results.iterrows():
        sig_mark = '*' if row['significant'] else ''
        print(f"{row['Model A']} & {row['Model B']} & {row['RMSE A']:.4f} & {row['RMSE B']:.4f} & {row['t-statistic']:.3f} & {row['p-value']:.4f} & {row['adjusted threshold']:.4f} & {sig_mark} \\\\")
    
    print("\\bottomrule")
    print("\\end{tabularx}")
    print("\\end{table}")

Running 28 pairwise tests with Holm-Bonferroni correction
                      Model A                    Model B  t-statistic  \
26  ElasticNet_LR_Base_Random          ElasticNet_LR_Yeo     4.891260   
27         ElasticNet_LR_Base          ElasticNet_LR_Yeo     4.891260   
23   ElasticNet_LR_Yeo_Random         ElasticNet_LR_Base    -4.859602   
22   ElasticNet_LR_Yeo_Random  ElasticNet_LR_Base_Random    -4.859602   
18              LR_Yeo_Random   ElasticNet_LR_Yeo_Random     2.621358   
21              LR_Yeo_Random          ElasticNet_LR_Yeo     2.618367   
9                      LR_Yeo   ElasticNet_LR_Yeo_Random     2.618109   
12                     LR_Yeo          ElasticNet_LR_Yeo     2.616075   
5                     LR_Base         ElasticNet_LR_Base    -1.376145   
4                     LR_Base  ElasticNet_LR_Base_Random    -1.376145   
6                     LR_Base          ElasticNet_LR_Yeo     1.304125   
3                     LR_Base   ElasticNet_LR_Yeo_Random     1.303

In [38]:
# Load the model_results.pkl file
with open('strat_linreg_eval_metrics/model_results.pkl', 'rb') as f:
    loaded_model_metrics = pickle.load(f)

# Print the keys to see what models are stored
print("Models in the pickle file:")
for key in loaded_model_metrics.keys():
    print(f"- {key}")

Models in the pickle file:
- LR_Base
- LR_Yeo
- LR_Base_Random
- LR_Yeo_Random
- Sector_Consumer Discretionary_Base
- Sector_Consumer Discretionary_Yeo
- Sector_Consumer Discretionary_Base_Random
- Sector_Consumer Discretionary_Yeo_Random
- Sector_Consumer Staples_Base
- Sector_Consumer Staples_Yeo
- Sector_Consumer Staples_Base_Random
- Sector_Consumer Staples_Yeo_Random
- Sector_Energy_Base
- Sector_Energy_Yeo
- Sector_Energy_Base_Random
- Sector_Energy_Yeo_Random
- Sector_Financials_Base
- Sector_Financials_Yeo
- Sector_Financials_Base_Random
- Sector_Financials_Yeo_Random
- Sector_Health Care_Base
- Sector_Health Care_Yeo
- Sector_Health Care_Base_Random
- Sector_Health Care_Yeo_Random
- Sector_Industrials_Base
- Sector_Industrials_Yeo
- Sector_Industrials_Base_Random
- Sector_Industrials_Yeo_Random
- Sector_Information Technology_Base
- Sector_Information Technology_Yeo
- Sector_Information Technology_Base_Random
- Sector_Information Technology_Yeo_Random
- Sector_Materials_Base
-

In [39]:
import sys
import os

# Add the directory containing the visualization module to the Python path
viz_path = '/mnt/d/mas_venvs/masenv/viz_utils'
if viz_path not in sys.path:
    sys.path.append(viz_path)

# Import the module with the new name
from model_viz import plot_metrics_comparison, plot_residual_analysis, plot_feature_importance, plot_rmse_distribution



In [41]:
# Example of comparing multiple models
model_metrics = {
    'LR_Base': model_metrics['LR_Base'],
    'LR_Yeo': model_metrics['LR_Yeo'],
    'ElasticNet_LR_Base': model_metrics['ElasticNet_LR_Base'],
    'ElasticNet_LR_Yeo': model_metrics['ElasticNet_LR_Yeo']
}

# Create the comparison plot
plot_metrics_comparison(
    model_metrics,
    output_dir='output/metrics',
    metric_subset=['RMSE', 'R2', 'MAE']  # Optional: specify which metrics to plot
)

,MSE,MAE,R2,RMSE
Model,,,,
LR_Base,3.476302,1.450904,0.106595,1.864484
LR_Yeo,3.709846,1.471139,0.046575,1.926096
ElasticNet_LR_Base,3.845329,1.551427,0.011756,1.960951
ElasticNet_LR_Yeo,3.162359,1.415361,0.187278,1.778302


In [42]:
# Analyze a specific model's residuals
plot_residual_analysis(
    model_metrics=model_metrics,
    model_name='ElasticNet_LR_Yeo',  # Choose the model you want to analyze
    output_dir='output/residuals'
)

# You can analyze multiple models by calling the function for each one
for model_name in ['LR_Base', 'ElasticNet_LR_Base']:
    plot_residual_analysis(
        model_metrics=model_metrics,
        model_name=model_name,
        output_dir=f'output/residuals/{model_name}'
    )

In [44]:
# Example with a dictionary of feature importances
feature_importances = model_metrics['ElasticNet_LR_Yeo']['feature_importances']

plot_feature_importance(
    feature_importances=feature_importances,
    top_n=15,  # Show top 15 features
    output_dir='output/features'
)

# Alternative with DataFrame input
import pandas as pd
feature_df = pd.DataFrame({
    'Feature': list(feature_importances.keys()),
    'Importance': list(feature_importances.values())
})
plot_feature_importance(
    feature_importances=feature_df,
    top_n=20,
    output_dir='output/features'
)

KeyError: 'feature_importances'

In [45]:
# Plot RMSE distribution from your CV results file
plot_rmse_distribution(
    results_file='strat_linreg_eval_metrics/elasticnet_stratified_params.pkl',
    output_dir='output/cv_results'
)

,Dataset,Fold,RMSE
0,LR_Base,1,1.900175
1,LR_Base,2,1.991787
2,LR_Base,3,1.917544
3,LR_Base,4,1.857646
4,LR_Base,5,1.914036
5,LR_Yeo,1,1.726230
6,LR_Yeo,2,1.830957
7,LR_Yeo,3,1.759412
8,LR_Yeo,4,1.657172
9,LR_Yeo,5,1.586418


In [46]:
# This is just an example of how you could modify a function call with more parameters:
plot_metrics_comparison(
    model_metrics=model_metrics,
    output_dir='output/metrics',
    metric_subset=['RMSE', 'R2'],
    dpi=600,               # Higher resolution
    figsize=(12, 8),       # Custom figure size
    output_format='pdf'    # Different output format
)

TypeError: plot_metrics_comparison() got an unexpected keyword argument 'dpi'

In [ ]:
# Import our custom modules
from permutation_importance import (
    load_data, 
    run_permutation_importance, 
    compare_feature_importances
)
from feature_importance_analysis import analyze_feature_importances
import pickle
import os

# Define custom paths
BASE_DIR = os.getcwd()

# Single pkl file path with all your data
MODEL_RESULTS_PATH = os.path.join(BASE_DIR, 'strat_linreg_eval_metrics', 'model_results.pkl')

# Path for data files (if needed)
FEATURE_PATH = os.path.join(BASE_DIR, 'data', 'combined_df_for_ml_models.csv')
SCORE_PATH = os.path.join(BASE_DIR, 'data', 'score.csv')

# Define output paths
OUTPUT_DIR = os.path.join(BASE_DIR, 'strat_linreg_eval_metrics')  # Custom output directory as requested
PERMUTATION_DIR = os.path.join(OUTPUT_DIR, 'permutation_importance')
ANALYSIS_DIR = os.path.join(PERMUTATION_DIR, 'analysis')
COMPARISON_DIR = os.path.join(PERMUTATION_DIR, 'comparisons')

# Ensure directories exist
os.makedirs(PERMUTATION_DIR, exist_ok=True)
os.makedirs(ANALYSIS_DIR, exist_ok=True)
os.makedirs(COMPARISON_DIR, exist_ok=True)

# Load the single pickle file containing all data
with open(MODEL_RESULTS_PATH, 'rb') as f:
    results_data = pickle.load(f)

# Extract the required data from your pickle file
# This depends on how your data is structured. Here's an example:
model_metrics = results_data['model_metrics']  # Adjust keys based on your actual structure
data_dict = results_data['data_dict']
base_columns = results_data['base_columns']
yeo_columns = results_data['yeo_columns']

# Define models to analyze
models_to_analyze = [
    'LR_Base',
    'LR_Yeo',
    'LR_Base_elasticnet',
    'LR_Yeo_elasticnet'
]

# Define custom model-dataset mapping using data from your pkl
custom_model_datasets = {
    'LR_Base': data_dict['LR_Base'],
    'LR_Yeo': data_dict['LR_Yeo'],
    'LR_Base_elasticnet': data_dict['LR_Base'],
    'LR_Yeo_elasticnet': data_dict['LR_Yeo']
}

# Run permutation importance with the data from your single pkl file
importances = run_permutation_importance(
    model_metrics=model_metrics,
    models_to_analyze=models_to_analyze,
    data_dict=data_dict,
    model_datasets=custom_model_datasets,
    output_dir=PERMUTATION_DIR,
    elasticnet_params_path=None,  # Not needed if params are in your single pkl file
    n_repeats=10,
    max_features=20
)

# Run model comparisons
compare_feature_importances(
    all_importances=importances,
    output_dir=COMPARISON_DIR
)

# Run comprehensive analysis
importance_df = analyze_feature_importances(
    model_metrics_path=MODEL_RESULTS_PATH,  # Your single pkl file path
    output_dir=ANALYSIS_DIR
)

# Save the updated results back to the original path
results_data['model_metrics'] = model_metrics  # Update the model_metrics component
with open(MODEL_RESULTS_PATH, 'wb') as f:
    pickle.dump(results_data, f)

print(f"Analysis complete. All results saved to {OUTPUT_DIR}/")